In [1]:
import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
import PyPDF2

c:\Users\mayur\MCQGen\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
KEY = os.getenv("GEMINI_KEY")

In [4]:
llm = ChatGoogleGenerativeAI(google_api_key = KEY, model = "gemini-pro", temperature=0.5)

In [239]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  json  and use it as a guide. \
Ensure to make {number} MCQs
{response_json}

"""

In [240]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [241]:
quize_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [242]:
quiz_chain = LLMChain(llm = llm, prompt=quize_generation_prompt, output_key="quiz", verbose=True)

In [243]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.Don't add any headings.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [244]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE2
)

In [245]:
review_chain = LLMChain(llm = llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [246]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain,review_chain], 
input_variables = ["text", "number", "subject", "tone", "response_json"], output_variables=["quiz","review"], verbose=True)  

In [247]:
with open("data.txt", "r") as file:
    TEXT = file.read()

In [248]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [249]:
# serialize the python dictionary into a JSON-FORMATTED String
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [250]:
NUMBER = 5
SUBJECT = "data science"
TONE = "simple"

In [251]:
# How to setup Token Usage Tracking in Langchain
response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON) 
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [252]:
print(f"Total number of tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion tokens: {cb.completion_tokens}")
print(f"Total cost: {cb.total_cost}")

Total number of tokens: 0
Prompt tokens: 0
Completion tokens: 0
Total cost: 0.0


In [253]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [254]:
quiz = response.get('quiz')

In [257]:
quiz = json.loads(quiz)

In [262]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )

    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [263]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Warren McCulloch | d: Tom M. Mitchell',
  'Correct': 'b'},
 {'MCQ': "What was the purpose of the 'Cybertron' machine?",
  'Choices': 'a: To play checkers | b: To analyze sonar signals | c: To recognize characters | d: To teach artificial intelligence',
  'Correct': 'b'},
 {'MCQ': "Which psychologist's work influenced the development of artificial neural networks?",
  'Choices': 'a: Walter Pitts | b: Donald Hebb | c: Alan Turing | d: Tom M. Mitchell',
  'Correct': 'b'},
 {'MCQ': 'What is a key objective of modern-day machine learning?',
  'Choices': 'a: To understand human cognitive processes | b: To classify data | c: To create artificial intelligence | d: To solve complex mathematical equations',
  'Correct': 'b'},
 {'MCQ': "According to Tom M. Mitchell's definition, what is the purpose of a machine learning algorithm?",
  'Choices': 'a: To improve its performance over time | b: To

In [265]:
quiz = pd.DataFrame(data = quiz_table_data)
data

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a: Donald Hebb | b: Arthur Samuel | c: Warren ...,b
1,What was the purpose of the 'Cybertron' machine?,a: To play checkers | b: To analyze sonar sign...,b
2,Which psychologist's work influenced the devel...,a: Walter Pitts | b: Donald Hebb | c: Alan Tur...,b
3,What is a key objective of modern-day machine ...,a: To understand human cognitive processes | b...,b
4,"According to Tom M. Mitchell's definition, wha...",a: To improve its performance over time | b: T...,a


In [266]:
quiz.to_csv("machine_learning_quiz.csv", index = False)